<a href="https://colab.research.google.com/github/Hyemmingway/ai_teamplay/blob/main/mp%20yes%20models%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 모듈 불러오기

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2
#import mediapipe as mp

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1. 데이터 확인

In [13]:
#한 폴더당 중심 동작 20개씩 뽑음
select_df=pd.read_csv('/content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 data/DF_mp_select_20_oversampling.csv')
select_df

,name,fold,class,num,path,label
0,00_01.0046.jpg,00_01,1,46,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
1,00_01.0047.jpg,00_01,1,47,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
2,00_01.0048.jpg,00_01,1,48,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
3,00_01.0049.jpg,00_01,1,49,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
4,00_07.0035.jpg,00_07,7,35,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,me
...,...,...,...,...,...,...
2655,00_13.0050.jpg,00_13,13,50,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,thank
2656,08_13.0071.jpg,08_13,13,71,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,thank
2657,05_13.0021.jpg,05_13,13,21,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,thank
2658,14_13.0073.jpg,14_13,13,73,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,thank


In [14]:
datagen = ImageDataGenerator(rescale=1/ 255, validation_split=0.3,
                             rotation_range=10, width_shift_range=0.1, 
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, vertical_flip=True)

train_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

Found 1862 validated image filenames belonging to 10 classes.
Found 798 validated image filenames belonging to 10 classes.


### CNN

In [20]:
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow import keras

In [21]:
from keras.callbacks import ModelCheckpoint

In [22]:
pool_size = 2
    
model1 = Sequential([
  keras.Input(shape=(256,256,3)),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),                                           
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Flatten(),
  Dense(50),
  Activation('relu'),                                          
  
  Dense(10, activation='softmax'),
])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 256, 256, 30)      840       
                                                                 
 batch_normalization_12 (Bat  (None, 256, 256, 30)     120       
 chNormalization)                                                
                                                                 
 activation_14 (Activation)  (None, 256, 256, 30)      0         
                                                                 
 conv2d_13 (Conv2D)          (None, 256, 256, 30)      8130      
                                                                 
 batch_normalization_13 (Bat  (None, 256, 256, 30)     120       
 chNormalization)                                                
                                                                 
 activation_15 (Activation)  (None, 256, 256, 30)     

In [23]:
model1.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [30]:
checkpoint_filepath = '/content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 data' + 'best_model1_{val_accuracy:.4f}.ckpt'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [25]:
#cnn
history1= model1.fit(
    train_generator,
    epochs=15,
    validation_data=valid_generator,
    shuffle=True)

Epoch 1/15
15/15 [==============================] - 45s 3s/step - loss: 22.0734 - accuracy: 0.1241 - val_loss: 2.5020 - val_accuracy: 0.1115
Epoch 2/15
15/15 [==============================] - 45s 3s/step - loss: 3.6678 - accuracy: 0.0999 - val_loss: 2.3027 - val_accuracy: 0.1190
Epoch 3/15
15/15 [==============================] - 42s 3s/step - loss: 2.3066 - accuracy: 0.1101 - val_loss: 2.3046 - val_accuracy: 0.0451
Epoch 4/15
15/15 [==============================] - 43s 3s/step - loss: 2.3014 - accuracy: 0.1235 - val_loss: 2.3067 - val_accuracy: 0.0451
Epoch 5/15
15/15 [==============================] - 42s 3s/step - loss: 2.3005 - accuracy: 0.1235 - val_loss: 2.3090 - val_accuracy: 0.0451
Epoch 6/15
15/15 [==============================] - 42s 3s/step - loss: 2.2996 - accuracy: 0.1235 - val_loss: 2.3114 - val_accuracy: 0.0451
Epoch 7/15
15/15 [==============================] - 41s 3s/step - loss: 2.2987 - accuracy: 0.1235 - val_loss: 2.3139 - val_accuracy: 0.0451
Epoch 8/15
15/15 [=

### CNN + dropout

In [26]:
pool_size = 2
    
model3 = Sequential([
  keras.Input(shape=(256,256,3)),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),                                           
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Flatten(),
  Dense(50),
  Activation('relu'),                                          
  Dropout(0.5),
  Dense(10, activation='softmax'),
])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 256, 256, 30)      840       
                                                                 
 batch_normalization_12 (Bat  (None, 256, 256, 30)     120       
 chNormalization)                                                
                                                                 
 activation_14 (Activation)  (None, 256, 256, 30)      0         
                                                                 
 conv2d_13 (Conv2D)          (None, 256, 256, 30)      8130      
                                                                 
 batch_normalization_13 (Bat  (None, 256, 256, 30)     120       
 chNormalization)                                                
                                                                 
 activation_15 (Activation)  (None, 256, 256, 30)     

In [27]:
model3.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [31]:
checkpoint_filepath = '/content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 data' + 'best_model3_{val_accuracy:.4f}.ckpt'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [29]:
#cnn
history3 = model3.fit(
    train_generator,
    epochs=15,
    validation_data=valid_generator,
    shuffle=True)

Epoch 1/15
15/15 [==============================] - 49s 3s/step - loss: 9.2075 - accuracy: 0.0945 - val_loss: 2.3105 - val_accuracy: 0.0827
Epoch 2/15
15/15 [==============================] - 44s 3s/step - loss: 2.3009 - accuracy: 0.1198 - val_loss: 2.3064 - val_accuracy: 0.0451
Epoch 3/15
15/15 [==============================] - 46s 3s/step - loss: 2.3005 - accuracy: 0.1208 - val_loss: 2.3093 - val_accuracy: 0.0188
Epoch 4/15
15/15 [==============================] - 42s 3s/step - loss: 2.2993 - accuracy: 0.1348 - val_loss: 2.3123 - val_accuracy: 0.0188
Epoch 5/15
15/15 [==============================] - 41s 3s/step - loss: 2.2982 - accuracy: 0.1348 - val_loss: 2.3154 - val_accuracy: 0.0188
Epoch 6/15
15/15 [==============================] - 41s 3s/step - loss: 2.2970 - accuracy: 0.1348 - val_loss: 2.3185 - val_accuracy: 0.0188
Epoch 7/15
15/15 [==============================] - 42s 3s/step - loss: 2.2960 - accuracy: 0.1348 - val_loss: 2.3217 - val_accuracy: 0.0188
Epoch 8/15
15/15 [==

### CNN+LSTM

In [32]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, MaxPooling2D,\
                        Permute, TimeDistributed, Bidirectional,GRU, LeakyReLU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Reshape
from keras import models
from keras import layers
import tensorflow as tf

In [33]:
Model = Sequential()

Model.add(SeparableConv2D(12,(3,3),activation="relu",
                 input_shape=(256,256,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))

#
Model.add(SeparableConv2D(24,(3,3),
                 activation="relu",padding="same"))
Model.add(Dropout(0.2))
Model.add(MaxPooling2D((2,2)))

#
Model.add(TimeDistributed(Flatten()))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))

#
Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(10,activation="sigmoid"))

In [34]:
Model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [35]:
checkpoint_filepath = '/content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 data' + 'best_model2_{val_accuracy:.4f}.ckpt'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [36]:
history2= Model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)

Epoch 1/10
15/15 [==============================] - 69s 4s/step - loss: 2.2803 - accuracy: 0.1477 - val_loss: 2.3367 - val_accuracy: 0.0890
Epoch 2/10
15/15 [==============================] - 57s 4s/step - loss: 2.2247 - accuracy: 0.1751 - val_loss: 2.3168 - val_accuracy: 0.0840
Epoch 3/10
15/15 [==============================] - 56s 4s/step - loss: 2.0988 - accuracy: 0.2562 - val_loss: 2.3068 - val_accuracy: 0.1441
Epoch 4/10
15/15 [==============================] - 56s 4s/step - loss: 1.9758 - accuracy: 0.2938 - val_loss: 2.5517 - val_accuracy: 0.1040
Epoch 5/10
15/15 [==============================] - 56s 4s/step - loss: 1.8321 - accuracy: 0.3410 - val_loss: 2.6649 - val_accuracy: 0.1040
Epoch 6/10
15/15 [==============================] - 56s 4s/step - loss: 1.7105 - accuracy: 0.3899 - val_loss: 2.4983 - val_accuracy: 0.1479
Epoch 7/10
15/15 [==============================] - 56s 4s/step - loss: 1.5683 - accuracy: 0.4420 - val_loss: 2.7443 - val_accuracy: 0.0890
Epoch 8/10
15/15 [==

### googlenet + imceptionV

In [37]:

import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input #InceptionV3, Xception, InceptionResNetV2
#from keras.applications.imagenet_utils import preprocess_input #others
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization

from tensorflow import keras
from PIL import Image
import tensorflow as tf

from keras.callbacks import ModelCheckpoint

In [39]:

InceptionV3(
    include_top=True, # classification (softmax) 부분을 포함할 것인지 선택하는 것입니다.
    weights="imagenet", # imageNet으로 pre-trained 된 모델을 받을 것인지를 선택하는 것입니다. 만약 pre-trained 된 모델을 받길 원하지 않는다면 None으로 넣으면 됩니다.
    input_tensor=None, 
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

96112376/96112376 [==============================] - 5s 0us/step


In [40]:

base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg') #Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5

#base_model.summary()

net = Dense(units=10)(base_model.output)
net = Activation('softmax')(net)
model4 = Model(inputs=base_model.input, outputs=net)

model4.summary()

87910968/87910968 [==============================] - 5s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_118 (Conv2D)            (None, None, None,   864         ['input_6[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_119 (Batch  (None, None, None,   96         ['conv2d_118[0][0]']             
 Normalization)              

In [41]:

checkpoint_filepath = '/content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 data' + 'best_model4_{val_accuracy:.4f}.ckpt'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)
     

In [42]:
model4.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [43]:

history4 = model4.fit(
          train_generator,
          epochs=10,
          validation_data=valid_generator, 
          callbacks=[model_checkpoint_callback]
)
     

Epoch 1/10
15/15 [==============================] - ETA: 0s - loss: 1.0721 - accuracy: 0.6412
Epoch 1: val_accuracy improved from -inf to 0.08521, saving model to /content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 databest_model4_0.0852.ckpt
15/15 [==============================] - 73s 4s/step - loss: 1.0721 - accuracy: 0.6412 - val_loss: 2.6110 - val_accuracy: 0.0852
Epoch 2/10
15/15 [==============================] - ETA: 0s - loss: 0.3036 - accuracy: 0.8947
Epoch 2: val_accuracy did not improve from 0.08521
15/15 [==============================] - 46s 3s/step - loss: 0.3036 - accuracy: 0.8947 - val_loss: 7.2139 - val_accuracy: 0.0727
Epoch 3/10
15/15 [==============================] - ETA: 0s - loss: 0.2137 - accuracy: 0.9345
Epoch 3: val_accuracy did not improve from 0.08521
15/15 [==============================] - 44s 3s/step - loss: 0.2137 - accuracy: 0.9345 - val_loss: 7.7222 - val_accuracy: 0.0815
Epoch 4/10
15/15 [==============================] - ETA: 0s - loss: 0.2058 - accurac